In [1]:
##!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import polars as pl
import duckdb as dd
from datetime import datetime

In [3]:
config_dict = {'train_csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
config_dict['test_csv_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
config_dict['train_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
config_dict['test_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [4]:
train_data = pl.read_csv(config_dict['train_csv_path'], low_memory=True)
train_data.describe()

statistic,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,…,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
str,str,str,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,…,f64,f64,f64,f64,f64,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64
"""count""","""3960""","""3960""",3960.0,3960.0,"""2555""",2421.0,"""3310""",3022.0,3027.0,3076.0,898.0,2954.0,2967.0,2954.0,"""1308""",743.0,740.0,740.0,"""3346""",2322.0,2282.0,1074.0,1062.0,1074.0,1063.0,2310.0,2271.0,2305.0,2267.0,2307.0,2269.0,2324.0,2285.0,"""2145""",1991.0,1991.0,…,1991.0,1991.0,1991.0,1991.0,1991.0,"""475""",475.0,"""1721""",1721.0,"""2736""",2733.0,2734.0,2731.0,2731.0,2729.0,2732.0,2729.0,2730.0,2730.0,2733.0,2734.0,2731.0,2729.0,2732.0,2730.0,2728.0,2725.0,2728.0,2730.0,2733.0,2736.0,"""2618""",2609.0,2606.0,"""3540""",3301.0,2736.0
"""null_count""","""0""","""0""",0.0,0.0,"""1405""",1539.0,"""650""",938.0,933.0,884.0,3062.0,1006.0,993.0,1006.0,"""2652""",3217.0,3220.0,3220.0,"""614""",1638.0,1678.0,2886.0,2898.0,2886.0,2897.0,1650.0,1689.0,1655.0,1693.0,1653.0,1691.0,1636.0,1675.0,"""1815""",1969.0,1969.0,…,1969.0,1969.0,1969.0,1969.0,1969.0,"""3485""",3485.0,"""2239""",2239.0,"""1224""",1227.0,1226.0,1229.0,1229.0,1231.0,1228.0,1231.0,1230.0,1230.0,1227.0,1226.0,1229.0,1231.0,1228.0,1230.0,1232.0,1235.0,1232.0,1230.0,1227.0,1224.0,"""1342""",1351.0,1354.0,"""420""",659.0,1224.0
"""mean""",null,null,10.433586,0.372727,null,65.454771,null,19.331929,55.946713,89.038615,27.278508,69.648951,81.597236,116.983074,null,4.989233,7.37027,27.581081,null,11.25969,0.476337,22.420438,1.829567,23.518622,1.904045,5.579654,0.330251,8.694924,0.61888,8.805635,0.620097,9.252775,0.785558,null,2.651431,6.719826,…,33.17338,20.02299,67.301883,34.389466,53.998726,null,2.178853,null,2.58955,null,2.370655,2.177762,2.399854,0.839253,2.297545,1.06369,0.586295,1.24652,1.062637,1.304793,1.685443,0.244599,1.340051,1.035505,1.499634,1.452346,1.62789,1.613636,1.158974,0.943652,27.896199,null,41.088923,57.763622,null,1.060588,0.580409
"""std""",null,null,3.574648,0.483591,null,22.341862,null,5.113934,7.473764,44.56904,5.567287,13.611226,13.665196,17.061225,null,2.014072,3.189662,17.707751,null,11.807781,0.499549,10.833995,0.612585,11.148951,0.612344,7.390161,0.470407,3.429301,0.485769,3.422167,0.485469,2.988863,0.410525,null,1.028267,92.586325,…,56.272346,70.21561,108.705918,84.050607,129.362539,null,0.849476,null,0.783937,null,1.673312,1.697117,1.588807,1.195601,1.705218,1.268282,1.049355,1.342582,1.258797,1.331715,1.543074,0.522956,1.411156,1.301712,1.492929,1.4956,1.445622,1.529178,1.343661,1.18546,20.338853,null,10.427433,13.196091,null,1.094875,0.771122
"""min""","""00008ff9""","""Fall""",5.0,0.0,"""Fall""",25.0,"""Fall""",0.0,33.0,0.0,18.0,0.0,27.0,0.0,"""Fall""",0.0,0.0,0.0

In [5]:
#train_data.glimpse()

In [8]:
train_data.columns

['id',
 'Basic_Demos-Enroll_Season',
 'Basic_Demos-Age',
 'Basic_Demos-Sex',
 'CGAS-Season',
 'CGAS-CGAS_Score',
 'Physical-Season',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Waist_Circumference',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'Fitness_Endurance-Season',
 'Fitness_Endurance-Max_Stage',
 'Fitness_Endurance-Time_Mins',
 'Fitness_Endurance-Time_Sec',
 'FGC-Season',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_GSND',
 'FGC-FGC_GSND_Zone',
 'FGC-FGC_GSD',
 'FGC-FGC_GSD_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'BIA-Season',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_BMC',
 'BIA-BIA_BMI',
 'BIA-BIA_BMR',
 'BIA-BIA_DEE',
 'BIA-BIA_ECW',
 'BIA-BIA_FFM',
 'BIA-BIA_FFMI',
 'BIA-BIA_FMI',
 'BIA-BIA_Fat',
 'BIA-BIA_Frame_num',
 'BIA-BIA_ICW',
 'BIA-BIA_LDM',
 'BIA-BIA_LST',
 'BIA-BIA_SMM',
 'BIA-BIA_TBW',
 'PAQ_A-Season',

In [10]:
train_data.filter(pl.col('id')=='00115b9f')

id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,…,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
str,str,i64,i64,str,i64,str,f64,f64,f64,f64,i64,i64,i64,str,i64,i64,i64,str,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,str,i64,f64,f64,…,f64,f64,f64,f64,f64,str,f64,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64,i64
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,30.4041,16.779,58.9338,26.4798,45.9966,null,null,"""Winter""",2.451,"""Summer""",4,2,4,0,5,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1


In [33]:
sample_parquet_file = config_dict['train_parquet_root_path']+'id=00115b9f'
parquet_sample_df = pl.read_parquet(sample_parquet_file)

In [12]:
parquet_sample_df.describe()

statistic,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0,43330.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",21664.5,-0.316384,0.016009,-0.16789,0.047388,-10.580416,0.0,42.29631,4053.578857,5.0462e13,4.470182,3.0,53.201683
"""std""",12508.437919,0.453665,0.502702,0.58571,0.106351,42.94717,0.0,208.168976,112.404037,1.9428e13,1.931421,0.0,14.244915
"""min""",0.0,-1.746094,-2.905339,-1.048372,0.0,-89.833092,0.0,0.0,3824.0,5.5000e10,1.0,3.0,41.0
"""25%""",10832.0,-0.684193,-0.30987,-0.649974,0.006431,-41.543835,0.0,2.392968,4028.666748,3.6890e13,3.0,3.0,42.0
"""50%""",21665.0,-0.366849,0.024974,-0.245365,0.023638,-15.084804,0.0,6.926829,4070.0,5.3480e13,5.0,3.0,50.0
"""75%""",32497.0,-0.010677,0.400677,0.20474,0.04142,12.220962,0.0,15.0,4147.0,6.6410e13,6.0,3.0,53.0
"""max""",43329.0,1.507865,1.666354,1.546979,4.004276,89.751656,0.0,2633.25,4188.5,8.6110e13,7.0,3.0,85.0


In [13]:
parquet_sample_df

step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
u32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i8,i8,f32
0,0.021536,0.022214,-1.02237,0.022853,-88.280762,0.0,53.0,4188.0,56940000000000,4,3,41.0
1,0.022005,0.022187,-1.01974,0.020231,-88.241707,0.0,51.666668,4188.166504,56945000000000,4,3,41.0
2,0.02224,0.022005,-1.019401,0.019893,-88.170067,0.0,50.333332,4188.333496,56950000000000,4,3,41.0
3,0.021589,0.022578,-1.018177,0.018667,-88.250031,0.0,50.5,4188.5,56955000000000,4,3,41.0
4,0.022005,0.023763,-1.014323,0.016848,-88.130775,0.0,33.166668,4181.0,57235000000000,4,3,41.0
…,…,…,…,…,…,…,…,…,…,…,…,…
43325,-0.008333,-0.02362,-1.006901,0.007224,-88.595741,0.0,0.0,3824.0,57580000000000,6,3,85.0
43326,0.04873,-0.076725,-0.953776,0.056839,-85.352219,0.0,0.0,3824.0,57640000000000,6,3,85.0
43327,0.38737,0.793151,-0.402214,0.069961,-24.097908,0.0,0.0,3824.0,57645000000000,6,3,85.0


In [30]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [35]:
parquet_sample_df = parquet_sample_df.with_columns([
    pl.lit('00115b9f').alias("id"),
    pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day"),
    pl.col("time_of_day").map_elements(get_mins, return_dtype=pl.Int32).alias("min_of_day"),
    pl.col("time_of_day").map_elements(get_secs, return_dtype=pl.Int32).alias("sec_of_day")
])

parquet_sample_df = parquet_sample_df.drop(['time_of_day'])

parquet_sample_df.head(10)

step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,weekday,quarter,relative_date_PCIAT,id,hour_of_day,min_of_day,sec_of_day
u32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,f32,str,i32,i32,i32
0,0.021536,0.022214,-1.02237,0.022853,-88.280762,0.0,53.0,4188.0,4,3,41.0,"""00115b9f""",18,40,0
1,0.022005,0.022187,-1.01974,0.020231,-88.241707,0.0,51.666668,4188.166504,4,3,41.0,"""00115b9f""",15,33,20
2,0.02224,0.022005,-1.019401,0.019893,-88.170067,0.0,50.333332,4188.333496,4,3,41.0,"""00115b9f""",12,26,40
3,0.021589,0.022578,-1.018177,0.018667,-88.250031,0.0,50.5,4188.5,4,3,41.0,"""00115b9f""",9,20,0
4,0.022005,0.023763,-1.014323,0.016848,-88.130775,0.0,33.166668,4181.0,4,3,41.0,"""00115b9f""",3,6,40
5,0.023281,0.025156,-1.015599,0.01668,-88.130775,0.0,31.333334,4181.0,4,3,41.0,"""00115b9f""",0,0,0
6,0.024818,0.027109,-1.015781,0.017497,-88.130775,0.0,29.5,4181.0,4,3,41.0,"""00115b9f""",20,53,20
7,0.022578,0.024766,-1.015026,0.015589,-88.14167,0.0,27.666666,4181.0,4,3,41.0,"""00115b9f""",17,46,40
8,0.022318,0.024792,-1.01599,0.016576,-88.130775,0.0,25.833334,4181.0,4,3,41.0,"""00115b9f""",14,40,0


In [36]:
train_data_sample = train_data.filter(pl.col('id')=='00115b9f')

sample_joined_features = dd.sql("select t1.*, t2.* exclude(id) \
from train_data_sample t1 join parquet_sample_df t2 on t1.id = t2.id").pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [37]:
sample_joined_features.head(10)

id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,…,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,weekday,quarter,relative_date_PCIAT,hour_of_day,min_of_day,sec_of_day
str,str,i64,i64,str,i64,str,f64,f64,f64,f64,i64,i64,i64,str,i64,i64,i64,str,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,str,i64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64,i64,u32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,f32,i32,i32,i32
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43329,0.792144,0.170486,-0.413108,0.063103,-16.771301,0.0,0.0,3824.0,6,3,85.0,5,46,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43328,0.801953,0.501589,-0.040937,0.045489,-2.113776,0.0,0.0,3824.0,6,3,85.0,8,53,20
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43327,0.38737,0.793151,-0.402214,0.069961,-24.097908,0.0,0.0,3824.0,6,3,85.0,12,0,0
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43326,0.04873,-0.076725,-0.953776,0.056839,-85.352219,0.0,0.0,3824.0,6,3,85.0,15,6,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43325,-0.008333,-0.02362,-1.006901,0.007224,-88.595741,0.0,0.0,3824.0,6,3,85.0,4,26,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43324,-0.008542,-0.023385,-1.006797,0.007117,-88.595741,0.0,0.0,3824.0,6,3,85.0,7,33,20
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43323,-0.005859,-0.024089,-1.007552,0.013274,-88.570198,0.0,0.0,3824.0,6,3,85.0,10,40,0
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292

In [38]:
sample_joined_features.shape

(43330, 97)

In [41]:
import os
actigraphy_file_list = []

for dirname, _, filenames in os.walk('/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet'):
    for filename in filenames:
        actigraphy_file_list.append(os.path.join(dirname, filename))

In [42]:
len(actigraphy_file_list)

996